## Content Based Recommender

Uses similarities between patterns based on angular distance of eudlidean distance of pattern features. Feature engineering for this recommender can have a large inpact on results.

look here wh

In [1]:
# import pandas
import pandas as pd
import numpy as np

# import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

from sklearn.metrics.pairwise import euclidean_distances

In [3]:
# import data
patterns = pd.read_csv('data/patterns_df.csv')

In [ ]:
# create column for just attribite soup or BOW

In [ ]:
# vectorize the words first (count vectorizor 

count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [ ]:
# # Compute the cosine similarity matrix
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
#TRY EUCLIDEAN DISTANCE AS WELL
# from lecture monday:

In [ ]:
from sklearn.preprocessing import StandardScaler

# Extract only some subset of columns to reduce computation time 
X = cars[['mpg', 'disp', 'hp', 'wt']].values

# Standardize the features so that no feature dominates the
# distance computations due to unit scale
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [ ]:
# Car the user is looking at
looking_at_car = [15, 300, 160, 3.2]
looking_at_car = scaler.transform([looking_at_car])

# Distance from all other cars
distances = euclidean_distances(X, looking_at_car)
distances = distances.reshape(-1)   # Before it was (n_cars, 1)

# Find the 3 indices with the minimum distance (highest similarity) to the car we're looking at
ordered_indices = distances.argsort()
closest_indices = ordered_indices[:3]

# Get the cars for these indices
closest_cars = cars.iloc[closest_indices]
closest_cars